In [1]:
import requests
import pandas as pd
import json
import time
import plotly.express as px
import statsmodels.api as sm

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

### Pulling Data From API

In [ ]:
url = "https://zillow56.p.rapidapi.com/search"

with open('api.txt', 'r') as file:
    api_key = file.read()

headers = {
	"X-RapidAPI-Key": api_key,
	"X-RapidAPI-Host": "zillow56.p.rapidapi.com"
}

lst_1 = []

for page in range(1, 21):
    querystring = {"location":"Utah County, UT", 
                "sortSelection":"days", 
                "page":f"{page}"}

    response = requests.get(url, headers=headers, params=querystring)

    data = response.json()
    df__ = pd.json_normalize(data['results'])
    lst_1.append(df__)
    time.sleep(4)

df = pd.concat(lst_1)

In [ ]:
# df.to_csv('zillow_orig.csv')

### Cleaning the Data

In [2]:
## Because I have limited API requests ##
df_orig = pd.read_csv('zillow_orig.csv')

In [5]:
print(df_orig.columns)

Index(['Unnamed: 0', 'bathrooms', 'bedrooms', 'city', 'country', 'currency',
       'daysOnZillow', 'group_type', 'homeStatus', 'homeStatusForHDP',
       'homeType', 'imgSrc', 'isFeatured', 'isNonOwnerOccupied',
       'isPreforeclosureAuction', 'isPremierBuilder', 'isShowcaseListing',
       'isUnmappable', 'isZillowOwned', 'latitude', 'livingArea', 'longitude',
       'newConstructionType', 'price', 'priceForHDP', 'priceSuffix',
       'providerListingID', 'rentZestimate', 'shouldHighlight', 'state',
       'streetAddress', 'unit', 'zipcode', 'zpid',
       'listing_sub_type.is_newHome', 'lotAreaUnit', 'lotAreaValue',
       'zestimate', 'taxAssessedValue', 'listing_sub_type.is_FSBA',
       'openHouse', 'listing_sub_type.is_openHouse',
       'open_house_info.open_house_showing', 'datePriceChanged', 'priceChange',
       'priceReduction'],
      dtype='object')


In [3]:
# Dropping columns of no interest
df = df_orig[['price', 'bathrooms', 'bedrooms', 'city', 'homeType', 'livingArea', 'zipcode', 'priceReduction', 'daysOnZillow']]
df = df[df['homeType'] != 'LOT']

In [7]:
df.columns

Index(['price', 'bathrooms', 'bedrooms', 'city', 'homeType', 'livingArea',
       'zipcode', 'priceReduction', 'daysOnZillow'],
      dtype='object')

### EDA

What is the relationship between bathrooms & price?  
What is the relationship between bedrooms & price?  
What is the relationship between the city & the price?  
What is the average price of a 3 bedroom, 2 bathroom home in Spanish Fork?  

In [4]:
df_avg_price_per_city = df.groupby('city').median(numeric_only = True).reset_index().sort_values('price', ascending = False)
fig = px.bar(df_avg_price_per_city, x = 'city', y = 'price')
fig.show()
fig.write_image('bar_chart_preliminary.png')


In [37]:
one_hot_encoded = pd.get_dummies(df['city'], drop_first = True)
df_prepared = pd.concat([df, one_hot_encoded], axis=1)

one_hot_encoded = pd.get_dummies(df_prepared['homeType'], drop_first = True)
df_prepared = pd.concat([df_prepared, one_hot_encoded], axis=1)

df_prepared = df_prepared.drop(columns = ['city', 'homeType', 'priceReduction', 'zipcode'])
df_prepared = df_prepared[-df_prepared['livingArea'].isna()]

model = sm.OLS(df_prepared['price'], df_prepared.drop(columns = 'price')).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                  price   R-squared (uncentered):                   0.851
Model:                            OLS   Adj. R-squared (uncentered):              0.842
Method:                 Least Squares   F-statistic:                              92.76
Date:                Wed, 08 Nov 2023   Prob (F-statistic):                   2.72e-186
Time:                        21:16:23   Log-Likelihood:                         -7658.7
No. Observations:                 534   AIC:                                  1.538e+04
Df Residuals:                     503   BIC:                                  1.551e+04
Df Model:                          31                                                  
Covariance Type:            nonrobust                                                  
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
bathrooms         1.527e+05   2.73e+04      5.585      0.000     9.9e+04    2.06e+05
bedrooms         -1.095e+04   2.26e+04     -0.486      0.627   -5.53e+04    3.34e+04
livingArea         213.0015     15.828     13.457      0.000     181.904     244.099
daysOnZillow      9774.9552   2826.899      3.458      0.001    4220.971    1.53e+04
American Fork    -5.295e+05   9.11e+04     -5.811      0.000   -7.09e+05   -3.51e+05
Cedar Hills      -4.392e+05   2.59e+05     -1.697      0.090   -9.48e+05    6.93e+04
Draper           -3.951e+05   2.28e+05     -1.733      0.084   -8.43e+05    5.27e+04
Eagle Mountain   -7.114e+05   9.46e+04     -7.519      0.000   -8.97e+05   -5.26e+05
Elk Ridge        -7.932e+05   4.33e+05     -1.833      0.067   -1.64e+06    5.68e+04
Genola           -3.463e+05   4.34e+05     -0.799      0.425    -1.2e+06    5.06e+05
Goshen           -9.194e+05   3.14e+05     -2.929      0.004   -1.54e+06   -3.03e+05
Heber City       -5.514e+05   1.62e+05     -3.397      0.001    -8.7e+05   -2.32e+05
Highland         -6.126e+05   1.65e+05     -3.712      0.000   -9.37e+05   -2.88e+05
Lehi             -6.698e+05   9.51e+04     -7.044      0.000   -8.57e+05   -4.83e+05
Lindon           -8.118e+05   1.94e+05     -4.176      0.000   -1.19e+06    -4.3e+05
Mapleton         -7.699e+05   1.22e+05     -6.312      0.000   -1.01e+06    -5.3e+05
Orem             -5.598e+05   9.75e+04     -5.739      0.000   -7.51e+05   -3.68e+05
Payson           -5.285e+05   1.23e+05     -4.300      0.000    -7.7e+05   -2.87e+05
Pleasant Grove   -5.475e+05   1.36e+05     -4.014      0.000   -8.15e+05    -2.8e+05
Provo            -4.944e+05   9.93e+04     -4.981      0.000   -6.89e+05   -2.99e+05
Salem            -2.428e+05    1.3e+05     -1.863      0.063   -4.99e+05    1.33e+04
Santaquin        -5.796e+05   1.45e+05     -3.998      0.000   -8.64e+05   -2.95e+05
Saratoga Springs -6.173e+05   8.69e+04     -7.100      0.000   -7.88e+05   -4.46e+05
Spanish Fork     -6.455e+05   1.19e+05     -5.404      0.000    -8.8e+05   -4.11e+05
Springville       -4.54e+05   1.13e+05     -4.010      0.000   -6.76e+05   -2.32e+05
Vineyard          -5.76e+05   1.67e+05     -3.456      0.001   -9.03e+05   -2.49e+05
Woodland Hills    9.147e+05   2.64e+05      3.462      0.001    3.96e+05    1.43e+06
MANUFACTURED      6.099e+04   3.13e+05      0.195      0.845   -5.53e+05    6.75e+05
MULTI_FAMILY      2.248e+05   1.43e+05      1.568      0.118   -5.69e+04    5.06e+05
SINGLE_FAMILY     1.938e+05   8.84e+04      2.192      0.029    2.01e+04    3.67e+05
TOWNHOUSE         1.821e+05   9.36e+04      1.947      0.052   -1657.277    3.66e+05
==============================================================================
Omnibus:       